In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

In [3]:
sufixes = [
    "St", "Ave", "Blvd", "Dr", "Rd", "Ln", 
    "Ct", "Cir", "Pl", "Way", "Ter", "Trl", 
    "Pkwy", "Plz", "Sq", "Loop", "Cres", "Aly", 
    "Hwy", "Path", "Expy", "Grv", "Hts", 
    "Mdws", "Pass", "Row", "Run", "Vw", "Walk", "Blf", 
    "Brk", "Crk", "Xing", "Dale", "Fld", 
    "Gld", "Hts", "Holw", "Isle", "Knl", 
    "Lndg", "Mill", "Nook", "Ovps", "Park", 
    "Pnes", "Pt", "Rdg", "Spgs", "Vis", 
    "Wd", "Mews", "Cl", "Esp", "Gdns", 
    "Grg", "Grn", "Mall", "Qy", "Vly", "Hbr",
    "Cv", "Bay", "Ests", "Trn", "Mnr" ,'(SAC', ' St'
]


roads = pd.read_excel('MapBookRoadIndex.xlsx')
roads = roads[roads['Road Name'] != 'Road Name']

roads['Road Name'] = roads['Road Name'].str.split(' ')

roads['Road Name'] = roads['Road Name'].apply(lambda x: x[:-1] if isinstance(x, list) else x)

def remove_suffix(words):
    if isinstance(words, list) and len(words) > 0:
        if words[-1] in sufixes:
            words = words[:-1]
    return words

roads['Road Name'] = roads['Road Name'].apply(remove_suffix)
roads['Road Name'] = roads['Road Name'].dropna()
roads['Road Name'] = roads['Road Name'].apply(remove_suffix)
roads['Road Name'] = roads['Road Name'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

roads = roads.dropna(subset=['Road Name'])


def remove_suffix(road_name, sufixes):
    for suffix in sufixes:
        
        if road_name.lower().endswith(suffix.lower()):
            road_name = road_name[:-len(suffix)].strip()
    return road_name

roads['Road Name'] = roads['Road Name'].apply(lambda x: remove_suffix(x, sufixes))
roads = roads.drop_duplicates(subset=['Road Name'])
roads = roads.reset_index(drop= True)
roads['Road Name'].iloc[0] = '1ST'
roads['Road Name'].iloc[2] = '3RD'

# please make sure to count that the number of streets in the data frame is equal to the number of streets
# to the number of streets in the excel file

In [32]:
service = Service(executable_path='/Users/colefirkins/Desktop/selenium_project/chromedriver')
#driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, timeout=2)



land_use_code = []
parcel_id = []
lot_size = []
tax_district = []
owner_name = []
tax_class = []
land_value = []
ag_value = []
structures_value = []


for street in roads['Road Name']:     
        try:
            
            driver.get('https://assessor.bernco.gov/public.access/Search/Disclaimer.aspx?FromUrl=../search/commonsearch.aspx?mode=realprop')
            
            login_form = driver.find_element(By.XPATH, '//*[@id="btAgree"]')
            wait.until(lambda d : login_form.is_displayed())
            login_form.click()
            
            search_bar = driver.find_element(By.XPATH, '//*[@id="inpStreet"]')
            wait.until(lambda d : search_bar.is_displayed())
            time.sleep(2)
            search_bar.send_keys(street)
            search_bar.send_keys(Keys.RETURN)                                                 
            #time.sleep(3)
            # Find the number of entries
            number = driver.find_element(By.XPATH, '//*[@id="frmMain"]/table/tbody/tr/td/div/div/table[2]/tbody/tr/td/table/tbody/tr[3]/td/center/table[*]/tbody/tr/td[3]/font/b[2]').text
            number = int(number)
             
            # Click on the first parcel of the street
            element_I = driver.find_element(By.XPATH, f'//*[@id="searchResults"]/tbody/tr[1]')
            
            wait.until(lambda d : element_I.is_displayed())
            element_I.click()
            #time.sleep(3)
            # Define where the nav button is
            
            
            for i in range(number):
                
                time.sleep(3)
                
                # Grab: Land Use, Parcel ID, Lot Size, Tax District, Owner Name, Tax Class.
                land_use_code.append(driver.find_element(By.XPATH,'//*[@id="Real Property Attributes"]/tbody/tr[4]/td[2]').text)
                parcel_id.append(driver.find_element(By.XPATH,'//*[@id="datalet_header_row"]/td/h1/table/tbody/tr[1]/td[1]').text)
                lot_size.append(driver.find_element(By.XPATH, '//*[@id="Real Property Attributes"]/tbody/tr[3]/td[2]').text)           
                tax_district.append(driver.find_element(By.XPATH, '//*[@id="Class"]/tbody/tr[2]/td[2]').text)
                owner_name.append(driver.find_element(By.XPATH, '//*[@id="Ownership for Tax Year Selected"]/tbody/tr[2]/td[2]').text)                            
                tax_class.append(driver.find_element(By.XPATH, '//*[@id="Class"]/tbody/tr[2]/td[2]').text)                                       
                time.sleep(3)
               
                # Navigate to the values page.
                values_page = driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[2]/a')
                wait.until(lambda d : values_page.is_displayed())
                values_page.click()
                
                # Grab: Land Value, Ag Value, Structures Value,
                land_value.append(driver.find_element(By.XPATH, '//*[@id="Values"]/tbody/tr[3]/td[2]').text)           
                ag_value.append(driver.find_element(By.XPATH, '//*[@id="Values"]/tbody/tr[4]/td[2]').text)
                structures_value.append(driver.find_element(By.XPATH, '//*[@id="Values"]/tbody/tr[4]/td[2]').text)
                #time.sleep(3)
                # Navigate back to the profile page
                profile_page = driver.find_element(By.XPATH,'//*[@id="sidemenu"]/ul/li[1]/a/span')
                wait.until(lambda d : profile_page.is_displayed())
                profile_page.click()
                #time.sleep(3)
             
                # Click to the next parcel
                #driver.find_element(By.XPATH,'//*[@id="trTaxYear"]/td/table/tbody/tr[1]/td/table/tbody/tr/td[3]/nav').click()
                nav_button = driver.find_element(By.XPATH,'//*[@id="trTaxYear"]/td/table/tbody/tr[1]/td/table/tbody/tr/td[3]/nav')
                wait.until(lambda d : nav_button.is_displayed())
                nav_button.click()
                
                print(f'{i} of {number}, on {street} complete')
        except:
            pass

driver.quit()

0 of 164, on 1ST complete
1 of 164, on 1ST complete
2 of 164, on 1ST complete
3 of 164, on 1ST complete
4 of 164, on 1ST complete
5 of 164, on 1ST complete
6 of 164, on 1ST complete
7 of 164, on 1ST complete
8 of 164, on 1ST complete
9 of 164, on 1ST complete
10 of 164, on 1ST complete
11 of 164, on 1ST complete
12 of 164, on 1ST complete
13 of 164, on 1ST complete
14 of 164, on 1ST complete
15 of 164, on 1ST complete
16 of 164, on 1ST complete
17 of 164, on 1ST complete
18 of 164, on 1ST complete
19 of 164, on 1ST complete
20 of 164, on 1ST complete
21 of 164, on 1ST complete
22 of 164, on 1ST complete
23 of 164, on 1ST complete
24 of 164, on 1ST complete
25 of 164, on 1ST complete
26 of 164, on 1ST complete
27 of 164, on 1ST complete
28 of 164, on 1ST complete
29 of 164, on 1ST complete
30 of 164, on 1ST complete
31 of 164, on 1ST complete
32 of 164, on 1ST complete
33 of 164, on 1ST complete
34 of 164, on 1ST complete
35 of 164, on 1ST complete
36 of 164, on 1ST complete
37 of 164, 

In [30]:
driver.quit()

In [22]:
for i in [land_use_code,
parcel_id,
lot_size,
tax_district,
owner_name,
tax_class,
land_value,
ag_value, structures_value]:
    print(len(i))

0
0
0
0
0
0
0
0
0


In [6]:
df = pd.DataFrame()
df['land use code'] = land_use_code
df['parcel id'] = parcel_id
df['parcel id'] = df['parcel id'].str.split(' ').str[1]
df['lot_size'] = lot_size
df['tax_district'] = tax_district
df['owner_name'] = owner_name
df['tax_class'] = tax_class
#df['land_value'] = land_value
#df['ag_value'] = ag_value 
#df['structures_value'] = structures_value

In [7]:
df

,land use code,parcel id,lot_size,tax_district,owner_name,tax_class
0,BAR/LOUNGE,101405714528821205,.9143,A1AM,LA CORONA REAL LLC C/O EDWARD T GARCIA,A1AM
1,WAREHOUSE,101405722021233502,.163,A1AM,JOURNAL PUBLISHING CO,A1AM
2,RES 2 SFR ON PARCEL,101405722717932707,.1584,A1AM,PEREZ FERNANDO & SOLEDAD,A1AM
3,RESIDENTIAL IMPROVED,101405722818332708,.0697,A1AM,MORENO ANGELA,A1AM
4,VACANT COMMERCIAL,101405722919032709,.4017,A1AM,HOMEWISE INC,A1AM
5,MANUFACTURING/PROCESSING,101405723221633505,1.0859,A1AM,JOURNAL PUBLISHING CO,A1AM
6,OBY ONLY,101405723424533705,.488,A1AM,FIRST & IRON LLC,A1AM
7,OBY ONLY,101405724625433707,.2744,A1AM,HOMEWISE ORPHEUM LLC,A1AM
8,OFFICES LOW RISE (1-4 STY),101405727734510826,6.7657,A1AM,CITY OF ALBUQUERQUE,A1AM
9,LOW RISE APARTMENTS O4U (TO 3 STY),101405731044011922,1.003,A1AM,BERNALILLO COUNTY C/O COUNTY MANAGER,A1AM
